# Asyncio Event Loop in Action 

Having some basic knowledge about `asyncio` let's take a deeper look at event loop running our coroutines (basically awaitable functions in this context). To see this we will let every task update global dictionary and a simple counter.
Note that some executions in `asyncio` that are commented out, also work, but not in jupyter notebook, since jupyter starts event loop for us at its launch. To run this code outside jupyter, uncomment alternative executions. 

In [11]:
LOGGER = {}
RUNNING = 0


async def some_work(name, t):
    global LOGGER, RUNNING
    start = time.time()
    LOGGER.update({name: 'ONGOING'})
    RUNNING += 1
    await asleep(t)
    LOGGER.update({name: f'duration: {time.time()-start:.2f}'})
    RUNNING -= 1
    return name

Here make note that we have sandwitched awaitable action so it reports its timings to global variables. Now we will add another task that will report back to us, what is happening in the event loop every 2 seconds.

In [12]:
async def read_statuses():
    start_async = time.time()
    while True:
        print(f'uptime: {time.time() - start_async:.2f}')
        print(f'Events in the loop: {LOGGER}')
        print(f'Still running: {RUNNING}\n')

        if not RUNNING: # important to end this coroutine after everything else returned
            break

        await asleep(2)

    return 'reader'

Now we will run everything in one event loop.

In [14]:
tasks_to_complete = [some_work('cleaning', 3),
                     some_work('reading', 7),
                     some_work('writing', 10)]
tasks_to_complete.extend([read_statuses()]) # remember to add our reader to task list!

# outside jupyter:
# loop = asyncio.new_event_loop()
# asyncio.set_event_loop(loop)  # Python 3.10+ specific
# loop.run_until_complete(asyncio.gather(*tasks_to_complete))
# loop.close()

results = await asyncio.gather(*tasks_to_complete)

uptime: 0.00
Events in the loop: {'cleaning': 'ONGOING', 'reading': 'ONGOING', 'writing': 'ONGOING'}
Still running: 3

uptime: 2.00
Events in the loop: {'cleaning': 'ONGOING', 'reading': 'ONGOING', 'writing': 'ONGOING'}
Still running: 3

uptime: 4.00
Events in the loop: {'cleaning': 'duration: 3.00', 'reading': 'ONGOING', 'writing': 'ONGOING'}
Still running: 2

uptime: 6.00
Events in the loop: {'cleaning': 'duration: 3.00', 'reading': 'ONGOING', 'writing': 'ONGOING'}
Still running: 2

uptime: 8.01
Events in the loop: {'cleaning': 'duration: 3.00', 'reading': 'duration: 7.00', 'writing': 'ONGOING'}
Still running: 1

uptime: 10.01
Events in the loop: {'cleaning': 'duration: 3.00', 'reading': 'duration: 7.00', 'writing': 'duration: 10.00'}
Still running: 0



At the beginning we put four tasks in the loop. One of them spits out what is happening every two seconds. Other tasks at first has `status`: `ONGOING`, which means they have been started but since they hit `await` our program doesn't have to focus here, it can move to next task (or in other scenario it can redraw GUI so our application is not frozen because we wait for database response). Loop also does remember that every awaitable task at some point will get the response and will be ready to continue, so it revisits them to see if that is the case. After four seconds we can see that task that waited 3 seconds has already been revisited and executed (its status has changed). In real-life scenario we wouldn't know how long it will take but event loop will take care of it anyways. At some point in time every task has ended and we stop reading. The loop is still running but there is no task to execute. We can also inspect results from tasks but they ofcourse don't have to return anything.

In [15]:
print(results)

['cleaning', 'reading', 'writing', 'reader']
